### Setup

In [ ]:
from common import *

figure_saver = figure_saver(sub_directory="model_scores")

## Compare model performance scores

In [ ]:
exp_folders = [
    "lagged_lai_only",
    "lagged_vod_only",
    "15_most_important",
    "no_temporal_shifts",
    "lagged_fapar_only",
    "fapar_only",
    "lagged_sif_only",
    "sif_only",
    "all",
    "lai_only",
    "vod_only",
    "best_top_15",
]
print(len(exp_folders))
pprint(exp_folders)

In [ ]:
data = load_experiment_data(exp_folders, "model_scores")
test_r2s = {
    experiment_name_dict[exp]: exp_data["test_r2"] for exp, exp_data in data.items()
}
train_r2s = {
    experiment_name_dict[exp]: exp_data["train_r2"] for exp, exp_data in data.items()
}

In [ ]:
print(f"No lags: {test_r2s['CURR']:0.2f}")
print(f"All: {test_r2s['BEST15']:0.2f}")

In [ ]:
keys = list(test_r2s)
indices = np.argsort(list(test_r2s.values()))[::-1]

In [ ]:
indices

In [ ]:
pd.Series(test_r2s).iloc[indices]

In [ ]:
pd.Series(train_r2s).iloc[indices]

In [ ]:
df = pd.DataFrame(
    {
        "test R2": pd.Series(test_r2s).iloc[indices],
        "train R2": pd.Series(train_r2s).iloc[indices],
    }
)
df["train - test R2"] = df["train R2"] - df["test R2"]
df

#### Sort by train - test gap

In [ ]:
df.sort_values("train - test R2")

In [ ]:
comp = ("ALL", "BEST15")
print(f"val R2 ({comp[1]} → {comp[0]}): {test_r2s[comp[0]] - test_r2s[comp[1]]:0.3f}")

In [ ]:
comp = ("BEST15", "CURR")
print(f"val R2 ({comp[1]} → {comp[0]}): {test_r2s[comp[0]] - test_r2s[comp[1]]:0.3f}")

In [ ]:
fig, ax1 = plt.subplots(figsize=(7, 2.2))
ax1.plot(np.array(list(test_r2s.values()))[indices], linestyle="", marker="o", c="C0")
ax1.set_ylabel("validation " r"$\mathrm{R}^2$", color="C0")
ax1.tick_params(axis="y", labelcolor="C0")

ax2 = ax1.twinx()
ax2.plot(np.array(list(train_r2s.values()))[indices], linestyle="", marker="x", c="C1")
ax2.set_ylabel("train " r"$\mathrm{R}^2$", color="C1")
ax2.tick_params(axis="y", labelcolor="C1")

ax1.grid(alpha=0.4, linestyle="--")
ax1.set_xticks(list(range(len(test_r2s))))
_ = ax1.set_xticklabels(
    list(np.array(list(test_r2s))[indices]), rotation=45, ha="right"
)

figure_saver.save_figure(fig, "model_comp_scores")